In [24]:
import numpy as np
import  pandas  as pd
from pandas import DataFrame

In [25]:
#read excel
data_excel = pd.read_excel('/Users/chloe/Desktop/train_data.xlsx')
raw_features = data_excel['sentence']
raw_label = data_excel.iloc[:,[1,2,3,4,5]]

In [26]:
raw_features.shape

(2671,)

In [27]:
raw_label.shape

(2671, 5)

In [28]:
labels = []
id_nolabel = []
for i in range(len(raw_label)):
    if not pd.isnull(raw_label.iloc[i][0]):
        labels.append(0)
    elif not pd.isnull(raw_label.iloc[i][1]):
        labels.append(1)
    elif not pd.isnull(raw_label.iloc[i][2]):
        labels.append(2)
    elif not pd.isnull(raw_label.iloc[i][3]):
        labels.append(3)
    elif not pd.isnull(raw_label.iloc[i][4]):
        labels.append(4)
    else :id_nolabel.append(i)

In [29]:
#delete invalid data
train_features=raw_features.drop(id_nolabel)

In [30]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
labels = class_le.fit_transform(labels)

In [31]:
import jieba
#import self-defined dictionary for e-commerce live streaming
jieba.load_userdict('/Users/chloe/Desktop/自有词典.txt')

In [32]:
#create a function defining stopwords 
def stopwordslist():
    stopwords = [line.strip() for line in open('/Users/chloe/Desktop/baidu_stopwords.txt',encoding='UTF-8').readlines()]
    return stopwords

In [33]:
# split sentences into words with Jieba 
def seg_depart(sentence):
    # print("正在分词")
    sentence_depart = jieba.cut(sentence.strip())
    # create a stopwords list 
    stopwords = stopwordslist()
    outstr = ''
    # delete the stopwords 
    for word in sentence_depart:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr

train_features_fenci = [seg_depart(f.replace("\n", "")) for f in train_features]  

In [34]:
#vectorize the words with the tool Word2Vec

In [35]:
from gensim.models.word2vec import Word2Vec  
n_dim=60
min_count =1
w2vmodel=Word2Vec(vector_size=n_dim,min_count=min_count)

w2vmodel.build_vocab(train_features_fenci)       
w2vmodel.train(train_features_fenci,total_examples=w2vmodel.corpus_count,epochs=10) #train the classifier

def m_avgvec(words,w2vmodel):       #vectorize sentences with vectors of each word
    return pd.DataFrame([w2vmodel.wv[w]
                       for w in words if w in w2vmodel.wv]).agg('mean')

train_vec=pd.DataFrame([m_avgvec(a,w2vmodel) for a in train_features_fenci])

In [36]:
type(train_vec[0][0])

numpy.float64

In [37]:
type(labels[0])

numpy.int64

In [38]:
# save data

In [39]:
#save features and labels
train_vec.to_csv('/Users/chloe/Desktop/data.csv',index=False)
#x = pd.read_csv(open('C:/Users/baikunlong/Desktop/陈豆豆/datasets/CDD_data.csv'))
np.savetxt('/Users/chloe/Desktop/Label.csv', labels, delimiter = ',')
#y = np.loadtxt(open("C:/Users/baikunlong/Desktop/陈豆豆/datasets/CDD_label.csv","rb"),delimiter=",",skiprows=0)

In [40]:
#read data, split train and test data set

In [41]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [42]:
x = pd.read_csv(open('/Users/chloe/Desktop/data.csv'))
y = np.loadtxt(open('/Users/chloe/Desktop/Label.csv',"rb"),delimiter=",",skiprows=0)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0,test_size=0.2)

In [43]:
from sklearn import svm,metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc, classification_report

In [44]:
random_state = np.random.RandomState(0)

In [45]:
#train the SVM classifier

In [46]:
model = OneVsRestClassifier(svm.SVC(kernel='linear',probability=True,random_state=random_state,C=1000))
clt = model.fit(x_train,y_train)

In [47]:
#test the model

In [48]:
y_test_pred = clt.predict(x_test)
label_list = ['Authority','Scarcity','Liking','Emotional contagion','Guarantee']

In [49]:
ov_acc = metrics.accuracy_score(y_test_pred,y_test)
print("overall accuracy: %f"%(ov_acc))
print("===========================================")
acc_for_each_class = metrics.precision_score(y_test,y_test_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("===========================================")
avg_acc = np.mean(acc_for_each_class)
print("average accuracy:%f"%(avg_acc))
print("===========================================")
classification_rep = classification_report(y_test,y_test_pred,target_names=label_list)
print("classification report: \n",classification_rep)
print("===========================================")

overall accuracy: 0.768224
acc_for_each_class:
 [0.75539568 0.76       0.83185841 0.7961165  0.675     ]
average accuracy:0.763674
classification report: 
                      precision    recall  f1-score   support

          Authority       0.76      0.88      0.81       120
           Scarcity       0.76      0.69      0.72       110
             Liking       0.83      0.88      0.85       107
Emotional contagion       0.80      0.81      0.80       101
          Guarantee       0.68      0.56      0.61        97

           accuracy                           0.77       535
          macro avg       0.76      0.76      0.76       535
       weighted avg       0.76      0.77      0.76       535



# predict

In [84]:
#read excel
data_excel = pd.read_excel('/Users/chloe/Desktop/train_to_predict.xlsx')
test_raw_features = data_excel['sentence']
name = data_excel['name']

In [85]:
test_raw_features.shape

(6713,)

In [86]:
# split sentences 
test_features_fenci = [seg_depart(f.replace("\n", "")) for f in test_raw_features]  

In [87]:
#vectorization 
from gensim.models.word2vec import Word2Vec  
n_dim=60
min_count =1
w2vmodel=Word2Vec(vector_size=n_dim,min_count=min_count)

w2vmodel.build_vocab(test_features_fenci)       
w2vmodel.train(test_features_fenci,total_examples=w2vmodel.corpus_count,epochs=10) #training the classifier 

def m_avgvec(words,w2vmodel):       #vectorizze the sentences
    return pd.DataFrame([w2vmodel.wv[w]
                       for w in words if w in w2vmodel.wv]).agg('mean')
predict_vec=pd.DataFrame([m_avgvec(a,w2vmodel) for a in test_features_fenci])

In [88]:
#存
predict_vec.to_csv('/Users/chloe/Desktop/to_predict_csv.csv',index=False)

In [89]:
#read predict data
predict_data = pd.read_csv(open('/Users/chloe/Desktop/to_predict_csv.csv'))

In [90]:
predict_data.shape

(6713, 60)

In [91]:
y_test_pred = clt.predict(predict_data)

In [92]:
output = pd.DataFrame(columns = ['name' , 'sentence', 'Authority', 'Scarcity', 'Liking', 'Emotional contagion', 'Guarantee'])
print(output.shape)
for i in range(len(y_test_pred)):
    if int(y_test_pred[i])==0:
        A1i=1
    else: A1i=0
        
    if int(y_test_pred[i])==1:
        A2i=1
    else: A2i=0
        
    if int(y_test_pred[i])==2:
        A3i=1
    else: A3i=0
        
    if int(y_test_pred[i])==3:
        A4i=1
    else: A4i=0
        
    if int(y_test_pred[i])==4:
        A5i=1
    else: A5i=0
        
    tupi = name[i], test_raw_features[i], A1i, A2i, A3i, A4i, A5i
    output.loc[i] = tupi
    
print(output.shape)

(0, 7)
(6713, 7)


In [93]:
output.to_excel('/Users/chloe/Desktop/to_predict_output.xlsx',index=False)